In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
import pandas as pd
import os, sys
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
#Reading the dataset
car_sales_df = pd.read_csv('Resources/car_data_combined_validation.csv')
car_sales_df

,Index,Resale_Cat,model,fuel_type,vehicle_class,displ,cyl,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,4,Low,BMW 750i,Gasoline,large car,4.4,8,SemiAuto-8,4WD,5,16,25,19,4,No,459,12071,8731,7909
1,65,Low,FIAT 500L,Gasoline,station wagon,1.4,4,Man-6,2WD,6,25,33,28,7,Yes,312,14380,11467,4916
2,119,Low,JAGUAR XJ,Gasoline,large car,3.0,6,SemiAuto-8,2WD,9,18,27,21,5,No,414,3542,2553,1348
3,132,Low,NISSAN Leaf,Electricity,midsize car,0.0,0,Auto-1,2WD,10,126,101,114,10,Elite,0,13251,10458,14565
4,139,High,TOYOTA 4Runner,Gasoline,standard SUV,4.0,6,SemiAuto-5,4WD,6,17,21,18,4,No,478,104456,119597,130025
5,276,High,GMC Sierra 15,Gasoline,pickup,5.3,8,Auto-8,2WD,5,16,22,18,3,No,497,207299,204211,30244
6,296,High,JEEP Wrangler,Gasoline,small SUV,3.6,6,Auto-5,4WD,6,17,21,18,3,No,481,180991,179188,228293
7,287,High,HONDA Ridgeline,Gasoline,pickup,3.5,6,Auto-6,2WD,6,19,26,22,5,No,410,23666,32068,28486


In [4]:
car_sales = car_sales_df.copy()

car_sales_clean = car_sales.copy()
car_sales_clean = car_sales_clean.drop(['model','Index','displ','cyl'], axis=1)
 

len(car_sales_clean.columns)

15

In [5]:
car_sales_sliced = car_sales_clean.copy()
bins = (3,18,23,114)
group_names = ['low-mpg','medium-mpg','high-mpg']
car_sales_sliced['cmb_mpg'] = pd.cut(car_sales_sliced['cmb_mpg'],bins = bins,labels = group_names)


In [6]:
car_sales_sliced['cmb_mpg'].unique()

[medium-mpg, high-mpg, low-mpg]
Categories (3, object): [low-mpg < medium-mpg < high-mpg]

In [7]:
lb = LabelEncoder()
car_sales_encoded = car_sales_sliced.copy()
car_sales_encoded['cmb_mpg']= lb.fit_transform(car_sales_encoded['cmb_mpg'])
car_sales_encoded['Resale_Cat']= lb.fit_transform(car_sales_encoded['Resale_Cat'])    
car_sales_encoded['fuel_type']= lb.fit_transform(car_sales_encoded['fuel_type'])
car_sales_encoded['vehicle_class']= lb.fit_transform(car_sales_encoded['vehicle_class'])
car_sales_encoded['trans']= lb.fit_transform(car_sales_encoded['trans'])
car_sales_encoded['drive']= lb.fit_transform(car_sales_encoded['drive'])
car_sales_encoded['smartway']= lb.fit_transform(car_sales_encoded['smartway'])

car_sales_encoded.head()

,Resale_Cat,fuel_type,vehicle_class,trans,drive,smog_rating,city_mpg,hwy_mpg,cmb_mpg,Greenhouse_gas_score,smartway,comb_CO2,Sales_2016,Sales_2017,Sales_2018
0,1,1,0,6,1,5,16,25,2,4,1,459,12071,8731,7909
1,1,1,5,4,0,6,25,33,0,7,2,312,14380,11467,4916
2,1,1,0,6,0,9,18,27,2,5,1,414,3542,2553,1348
3,1,0,1,0,0,10,126,101,0,10,0,0,13251,10458,14565
4,0,1,4,5,1,6,17,21,1,4,1,478,104456,119597,130025


In [8]:
car_sales_encoded.to_csv('Resources/encoded_output.csv',index = False)

In [9]:
y_original = car_sales_encoded['cmb_mpg']
print(y_original)

0    2
1    0
2    2
3    0
4    1
5    1
6    1
7    2
Name: cmb_mpg, dtype: int64


In [10]:
# Sepearate dataset
car_sales_ml = car_sales_encoded.copy()
X = car_sales_ml.drop('cmb_mpg', axis=1)
y = car_sales_ml['cmb_mpg']

In [11]:
# Encode the dependent variable
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [12]:
#Define the encoder function
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode

In [13]:
y = encoder.transform(y)
Y = one_hot_encode(y)
sc = StandardScaler()
X = sc.fit_transform(X)

In [14]:


print(X.shape)
print(Y.shape)

print(X)
print(Y)

(8, 14)
(8, 3)
[[ 1.          0.37796447 -1.25667616  1.23959081  1.29099445 -0.94064699
  -0.44083978 -0.37391723 -0.51034318  0.          0.5026406  -0.74275012
  -0.79527629 -0.62911869]
 [ 1.          0.37796447  1.70020892  0.29514067 -0.77459667 -0.3617873
  -0.18893134 -0.05903956  0.85057197  2.         -0.44921986 -0.71312275
  -0.76035226 -0.66849925]
 [ 1.          0.37796447 -1.25667616  1.23959081 -0.77459667  1.37479175
  -0.38486013 -0.29519782 -0.0567048   0.          0.21125474 -0.85218791
  -0.87413617 -0.71544541]
 [ 1.         -2.64575131 -0.66529914 -1.59375961 -0.77459667  1.95365143
   2.63804125  2.61742064  2.21148711 -2.         -2.46949511 -0.72760924
  -0.77323177 -0.54154199]
 [-1.          0.37796447  1.10883191  0.76736574  1.29099445 -0.3617873
  -0.41284996 -0.53135607 -0.51034318  0.          0.62567018  0.44266552
   0.61988709  0.97762946]
 [-1.          0.37796447 -0.07392213 -0.1770844  -0.77459667 -0.94064699
  -0.44083978 -0.49199636 -0.96398156 

In [15]:
# Define the important parameters and variable 
#to work with the tensors

learning_rate = 0.3
training_epochs = 600
cost_history = np.empty(shape=[1],dtype=float)
n_dim = X.shape[1]
print('n_dim',n_dim)

n_dim 14


In [16]:
n_class = 3
model_path = "Model/tensorflow/NMI"

#Define the number of hidden layers and number of neurons for each layer 
n_hidden_1 = 14
n_hidden_2 = 10
n_hidden_3 = 10
n_hidden_4 = 8

x = tf.placeholder(tf.float32,[None,n_dim])
w = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32,[None, n_class])

In [17]:
#Define the model
def multilayer_perceptron(x,weights,biases):
    #Hidden layer with RELU activationsed
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    #Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    #Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)   
    
    #Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    #Output layer with linear activation
    output_layer = tf.matmul(layer_4,weights['out']+biases['out'])
    
    return output_layer
    

In [18]:
#Define the weights and the biases for each layer
weights = {
    'h1':tf.Variable(tf.truncated_normal([n_dim,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_4,n_class]))
}
biases = {
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_class]))
}

In [19]:
# Call the model defined
y = multilayer_perceptron(x, weights, biases)

In [20]:
# Define cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)


W0310 00:26:04.651309 140735700255616 deprecation.py:323] From <ipython-input-20-a06297189e3a>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [21]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

saver = tf.train.Saver()
saver.restore(sess,model_path)

W0310 00:26:05.033268 140735700255616 deprecation.py:323] From /Users/minaljajoo/anaconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [22]:
prediction = tf.argmax(y,1)
print(prediction)

Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [23]:
# Print the final accuracy
correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
print(tf.argmax(y,1))

Tensor("ArgMax_2:0", shape=(?,), dtype=int64)


In [24]:
accuracy = tf.reduce_mean(
    tf.cast(correct_prediction, tf.float32))
print(accuracy)
# Print Accuracy run
print('*****************************************************')
print('0 Stands for low-mpg, 1 stands for medium-mpg & 2 stands for high-mpg')
print('*****************************************************')

Tensor("Mean_1:0", shape=(), dtype=float32)
*****************************************************
0 Stands for low-mpg, 1 stands for medium-mpg & 2 stands for high-mpg
*****************************************************


In [25]:
print(X)

[[ 1.          0.37796447 -1.25667616  1.23959081  1.29099445 -0.94064699
  -0.44083978 -0.37391723 -0.51034318  0.          0.5026406  -0.74275012
  -0.79527629 -0.62911869]
 [ 1.          0.37796447  1.70020892  0.29514067 -0.77459667 -0.3617873
  -0.18893134 -0.05903956  0.85057197  2.         -0.44921986 -0.71312275
  -0.76035226 -0.66849925]
 [ 1.          0.37796447 -1.25667616  1.23959081 -0.77459667  1.37479175
  -0.38486013 -0.29519782 -0.0567048   0.          0.21125474 -0.85218791
  -0.87413617 -0.71544541]
 [ 1.         -2.64575131 -0.66529914 -1.59375961 -0.77459667  1.95365143
   2.63804125  2.61742064  2.21148711 -2.         -2.46949511 -0.72760924
  -0.77323177 -0.54154199]
 [-1.          0.37796447  1.10883191  0.76736574  1.29099445 -0.3617873
  -0.41284996 -0.53135607 -0.51034318  0.          0.62567018  0.44266552
   0.61988709  0.97762946]
 [-1.          0.37796447 -0.07392213 -0.1770844  -0.77459667 -0.94064699
  -0.44083978 -0.49199636 -0.96398156  0.          0.

In [36]:
pred_mpg = []
for i in range(0, 7):
    prediction_run = sess.run(prediction,
                              feed_dict={x: X[i].reshape(1, 14)})
    accuracy_run = sess.run(accuracy,
                            feed_dict={
                                x: X[i].reshape(1, 14),
                                y_: Y[i].reshape(1, 3)})
    print('Original label/class:', y_original[i],
          ' Predicted value:', prediction_run[0],
          ' Accuracy:', accuracy_run)
    pred_mpg.append(prediction_run[0])
    
print(pred_mpg)

Original label/class: 2  Predicted value: 1  Accuracy: 0.0
Original label/class: 0  Predicted value: 0  Accuracy: 1.0
Original label/class: 2  Predicted value: 2  Accuracy: 1.0
Original label/class: 0  Predicted value: 0  Accuracy: 1.0
Original label/class: 1  Predicted value: 1  Accuracy: 1.0
Original label/class: 1  Predicted value: 1  Accuracy: 1.0
Original label/class: 1  Predicted value: 1  Accuracy: 1.0
[1, 0, 2, 0, 1, 1, 1]


In [44]:
pred_df = pd.DataFrame(pred_mpg)
pred_df.to_csv("Model/pred_mpg_tf.csv", index=False)

In [27]:
# Print the final accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(
    tf.cast(correct_prediction, tf.float32))
print('Test Accuracy: ',
      (sess.run(accuracy, feed_dict ={x: X, y_: Y})))
print(correct_prediction)
print(accuracy)

Test Accuracy:  0.875
Tensor("Equal_1:0", shape=(?,), dtype=bool)
Tensor("Mean_2:0", shape=(), dtype=float32)


In [33]:
for node in sess.graph_def.node:
    print(node.name)

Placeholder
zeros
Variable
Variable/Assign
Variable/read
zeros_1
Variable_1
Variable_1/Assign
Variable_1/read
Placeholder_1
truncated_normal/shape
truncated_normal/mean
truncated_normal/stddev
truncated_normal/TruncatedNormal
truncated_normal/mul
truncated_normal
Variable_2
Variable_2/Assign
Variable_2/read
truncated_normal_1/shape
truncated_normal_1/mean
truncated_normal_1/stddev
truncated_normal_1/TruncatedNormal
truncated_normal_1/mul
truncated_normal_1
Variable_3
Variable_3/Assign
Variable_3/read
truncated_normal_2/shape
truncated_normal_2/mean
truncated_normal_2/stddev
truncated_normal_2/TruncatedNormal
truncated_normal_2/mul
truncated_normal_2
Variable_4
Variable_4/Assign
Variable_4/read
truncated_normal_3/shape
truncated_normal_3/mean
truncated_normal_3/stddev
truncated_normal_3/TruncatedNormal
truncated_normal_3/mul
truncated_normal_3
Variable_5
Variable_5/Assign
Variable_5/read
truncated_normal_4/shape
truncated_normal_4/mean
truncated_normal_4/stddev
truncated_normal_4/Trunca